In [1]:
import logging
import os

working_dir = "/global/cfs/cdirs/m4581/lee1043/work/cdat/pmp/mean_climate/mean_climate_workflow_refactorization/output"

logging.basicConfig(filename=os.path.join(working_dir, 'error_log.txt'), level=logging.ERROR, filemode='w',
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [2]:
from pcmdi_metrics.mean_climate.lib_unified import (
    get_unique_bases, process_dataset, calculate_and_save_metrics, extract_info_from_model_catalogue, 
    get_ref_catalogue, get_model_catalogue, 
    multi_level_dict, print_dict, write_to_json
)
import os
import datetime


from pcmdi_metrics.utils import create_target_grid

ERROR 1: PROJ: proj_create_from_database: Open of /global/homes/l/lee1043/.conda/envs/pmp_devel_20250722/share/proj failed


In [3]:
"""
variables = [
    "pr", "psl", 
    "ua-200", "ua-850", "va-200", "ta-850",
    "rsdt", "rsut", "rsutcs", "rlut", "rlutcs", 
    "rstcre", "rltcre", "rt", "rst"
]  # optional. If given, prioritized over the model_catalogue.json. If not given, use all variables commonly in ref_catalogue.json and model_catalogue.json

#variables = [
#    "ua-200", "ua-850", "va-200"
#]

#variables = ["psl"]
"""

model_data_path_template = "/home/data/%(model)/%(var)/%(model)_%(run)_%(var)_blabla.nc"  # optional. If given, prioritized over model_catalogue.json

models = ["model-a", "model-b"]  # optional. If given, prioritized over the model_catalogue.json. If not given, use all models in model_catalogue.json

models_runs_dict = {
    "model-a": ["r1", "r2"],
    "model-b": ["r1", "r2"],
    "model-c": ["r1", "r2"],
}
# optional. If given, prioritized over the model_catalogue.json. If not given, use all runs in model_catalogue.json

In [4]:
output_path = "/global/cfs/cdirs/m4581/lee1043/work/cdat/pmp/mean_climate/mean_climate_workflow_refactorization/output/json"

regions = ["NHEX", "SHEX"]

target_grid = "2.5x2.5"

ref_catalogue_file_path = '/global/cfs/projectdirs/m4581/obs4MIPs/catalogue/obs4MIPs_PCMDI_monthly_byVar_catalogue_v20250825.json'
model_catalogue_file_path = 'model_catalogue.json'

ref_data_head = "/global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL"  # optional, if ref_catalogue file does not include entire directory path


In [5]:
rad_diagnostic_variables = ["rt", "rst", "rstcre", "rltcre"]

default_regions = ["global", "NHEX", "SHEX", "TROPICS"]

In [6]:
if not regions:
    regions = default_regions

In [7]:
refs_dict = get_ref_catalogue(ref_catalogue_file_path, ref_data_head)
variables = sorted(list(refs_dict.keys()))

variables = [ 
 'ta',
 'tas',
 'tasmax',
 'tasmin',
 'tauu',
 'tauv',
 'ts',
 'ua',
 'uas',
 'va',
 'vas',
 'zg',
 'zos']

In [8]:
models_dict = get_model_catalogue(model_catalogue_file_path, variables, models, models_runs_dict, model_data_path_template)

In [9]:
import glob

In [10]:
era5_files = glob.glob("/global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ECMWF/ERA-5/mon/*/gn/latest/*.nc")

In [11]:
# List of files (replace this with your full list of files)
files = era5_files

# Initialize an empty dictionary to group files
grouped_files = {}

# Loop through each file path
for file in files:
    # Extract the variable name (e.g., vas, va, tas, etc.)
    variable = file.split('/')[-1].split('_')[0]
    
    # Extract the dataset name (e.g., ERA-5)
    dataset = file.split('/')[8]
    
    # Construct the template path with wildcard for the time range
    template = '/'.join(file.split('/')[:-1]) + f'/{variable}_mon_{dataset}_PCMDI_gn_*.nc'
    
    # Add to the dictionary
    if variable not in grouped_files:
        grouped_files[variable] = {}
    if dataset not in grouped_files[variable]:
        grouped_files[variable][dataset] = {"template": template}

# Print the resulting dictionary
import json
print(json.dumps(grouped_files, indent=4))

{
    "vas": {
        "ERA-5": {
            "template": "/global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ECMWF/ERA-5/mon/vas/gn/latest/vas_mon_ERA-5_PCMDI_gn_*.nc"
        }
    },
    "uas": {
        "ERA-5": {
            "template": "/global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ECMWF/ERA-5/mon/uas/gn/latest/uas_mon_ERA-5_PCMDI_gn_*.nc"
        }
    },
    "va": {
        "ERA-5": {
            "template": "/global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ECMWF/ERA-5/mon/va/gn/latest/va_mon_ERA-5_PCMDI_gn_*.nc"
        }
    },
    "tas": {
        "ERA-5": {
            "template": "/global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ECMWF/ERA-5/mon/tas/gn/latest/tas_mon_ERA-5_PCMDI_gn_*.nc"
        }
    },
    "ua": {
        "ERA-5": {
            "template": "/global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ECMWF/ERA-5/mon/ua/gn/latest/ua_mon_ERA-5_PCMDI_gn_*.nc"
        }
    },
    "ta": {
        "ERA-5": {
            "template": "/global/cfs/projec

In [12]:
era5_vars = sorted(list(grouped_files.keys()))

In [13]:
if any(var is None for var in (variables, models, models_runs_dict)): 
    variables, models, models_runs_dict = extract_info_from_model_catalogue(variables, models, models_runs_dict, refs_dict, models_dict)

In [14]:
common_grid = create_target_grid(target_grid_resolution=target_grid)

In [ ]:
encountered_variables = set()
ac_ref_dict = multi_level_dict()
ac_model_run_dict = multi_level_dict()
metrics_dict = multi_level_dict()



all_ref_variables = True

if all_ref_variables:
    variables = sorted(list(refs_dict.keys()))

variables_level_dict = get_unique_bases(variables)
variables_unique = list(variables_level_dict.keys())

"""
#target_ref = None
#target_ref = "CERES-EBAF-4-2"
target_ref = "ERA-5"

if target_ref == "ERA-5":
    variables_unique = era5_vars
    refs_dict = grouped_files
"""

print('variables_unique:', variables_unique)
print('variables_level_dict:', variables_level_dict)


def process_references(var, refs, rad_diagnostic_variables, levels, common_grid, start, end, version, interim_output_path_dict):
    for ref in refs:
        print(f"=== var, ref: {var}, {ref}")
        try:
        #if 1:
            process_dataset(
                var, ref, refs_dict, ac_ref_dict, rad_diagnostic_variables, 
                encountered_variables, levels, common_grid, interim_output_path_dict["ref"], data_type="ref",
                start=start, end=end,     
                repair_time_axis=True,
                overwrite_output_ac=True,
                version=version
            )
        
        except Exception as e:
            # Log the error to a file
            logging.error(f"Error for {var} {ref}: {str(e)}")
            print(f"Error logged for {var} {ref}")
            print(f'Error from process_references for {var} {ref}:', e)


def process_models(var, models, models_runs_dict, rad_diagnostic_variables, levels, common_grid, refs, start, end, version, interim_output_path_dict):
    for model in models:
        for run in models_runs_dict[model]:
            try:
                process_dataset(
                    var, (model, run), models_dict, ac_model_run_dict, 
                    rad_diagnostic_variables, encountered_variables, levels, common_grid, 
                    interim_output_path_dict["model"], data_type="model",
                    start=start, end=end,
                    version=version
                )
                for level in levels:
                    ac_model_run_level_interp = ac_model_run_dict[var][model][run][level]
                    calculate_and_save_metrics(var, model, run, level, regions, refs, ac_ref_dict, ac_model_run_level_interp, output_path, refs_dict, metrics_dict)
            except Exception as e:
                print(f'Error from process_models for {var} {model} {run}:', e)

    for level in levels:
        if level is None:
             var_key = var
        else:
             var_key = f"{var}-{level}"
        write_to_json(metrics_dict[var_key], os.path.join(output_path, f"output_{var_key}.json"))


def main():  
    # Set version identifier using the current date if not provided
    version = datetime.datetime.now().strftime("v%Y%m%d")
    version = "v20250825"
    print("version:", version)

    #start = "1981-01"
    #end = "2004-12"
    
    start = "1980-01"
    end = "2014-12"

    syear = start.split("-")[0]
    eyear = end.split("-")[0]

    interim_output_path_dict = {
        "ref": {
            "path_ac": f"/global/cfs/projectdirs/m4581/PMP/pmp_reference/obs4MIPs_clims_{syear}-{eyear}/%(var)/gn",
            "path_ac_interp": f"/global/cfs/projectdirs/m4581/PMP/pmp_reference/obs4MIPs_clims_{syear}-{eyear}/%(var)/gr"
        },
        "model": {
            "path_ac": f"/global/cfs/cdirs/m4581/lee1043/work/cdat/pmp/mean_climate/mean_climate_workflow_refactorization/output/clim_models_{syear}-{eyear}/%(var)/gn",
            "path_ac_interp": f"/global/cfs/cdirs/m4581/lee1043/work/cdat/pmp/mean_climate/mean_climate_workflow_refactorization/output/clim_models_{syear}-{eyear}/%(var)/gr"
        }
    }
    
    #variables_unique = ["pr"]
    #variables_unique = ["psl"]
    #variables_unique = ["ta", "ua", "va", "zg"]
    #variables_unique = ["tas", "ta"]
    #variables_unique = ["ta"]
    variables_unique.remove("pr")

    for var in variables_unique:
        try:
            print("var:", var)
            encountered_variables.add(var)
            levels = variables_level_dict[var]

            print("levels:", levels)

            if var in refs_dict:
                refs = refs_dict[var].keys()

                """
                if target_ref is not None:
                    if target_ref in refs:
                        refs = [target_ref]
                    else:
                        refs = []
                """
                process_references(var, refs, rad_diagnostic_variables, levels, common_grid, start, end, version, interim_output_path_dict)

                # process_models(var, models, models_runs_dict, rad_diagnostic_variables, levels, common_grid, refs, start, end, version)
        
        except Exception as e:
            print(f'Error from main for {var}:', e)            
        
if __name__ == "__main__":
    main()


variables_unique: ['hfls', 'hfns', 'hfss', 'hur', 'hus', 'pr', 'prw', 'psl', 'rlds', 'rldscs', 'rltcre', 'rlus', 'rlut', 'rlutcs', 'rsds', 'rsdscs', 'rsdt', 'rstcre', 'rsus', 'rsuscs', 'rsut', 'rsutcs', 'rt', 'sfcWind', 'ta', 'tas', 'tasmax', 'tasmin', 'tauu', 'tauv', 'ts', 'ua', 'uas', 'va', 'vas', 'zg', 'zos']
variables_level_dict: {'hfls': [None], 'hfns': [None], 'hfss': [None], 'hur': [None], 'hus': [None], 'pr': [None], 'prw': [None], 'psl': [None], 'rlds': [None], 'rldscs': [None], 'rltcre': [None], 'rlus': [None], 'rlut': [None], 'rlutcs': [None], 'rsds': [None], 'rsdscs': [None], 'rsdt': [None], 'rstcre': [None], 'rsus': [None], 'rsuscs': [None], 'rsut': [None], 'rsutcs': [None], 'rt': [None], 'sfcWind': [None], 'ta': [None], 'tas': [None], 'tasmax': [None], 'tasmin': [None], 'tauu': [None], 'tauv': [None], 'ts': [None], 'ua': [None], 'uas': [None], 'va': [None], 'vas': [None], 'zg': [None], 'zos': [None]}
version: v20250825
var: hfls
levels: [None]
=== var, ref: hfls, ERA-INT


2025-08-28 09:40:54,780 [ERROR]: 2831988338.py(process_references:46) >> Error for hfls ERA-INT: local variable 'levels_to_plot' referenced before assignment


regrid done, ds_ac_interp[hfls].shape: (12, 72, 144)
data_var, da.shape: hfls (12, 72, 144)
Error logged for hfls ERA-INT
Error from process_references for hfls ERA-INT: local variable 'levels_to_plot' referenced before assignment
=== var, ref: hfls, TropFlux-1-0
Processing data for: TropFlux-1-0
jwlee123, ref, data_path: TropFlux-1-0 /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ESSO/TropFlux-1-0/mon/hfls/gn/v20210727/hfls_mon_TropFlux-1-0_PCMDI_gn_197901-201707.nc
Processing ref dataset - varname: hfls, data: TropFlux-1-0, path: /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ESSO/TropFlux-1-0/mon/hfls/gn/v20210727/hfls_mon_TropFlux-1-0_PCMDI_gn_197901-201707.nc
ver: v20250828
ver: v20250828
get_annual_cycle, var: hfls
data_path: /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ESSO/TropFlux-1-0/mon/hfls/gn/v20210727/hfls_mon_TropFlux-1-0_PCMDI_gn_197901-201707.nc
out_path: /global/cfs/projectdirs/m4581/PMP/pmp_reference/obs4MIPs_clims_1980-2014/hfls/gn
outfilename_he

/global/homes/l/lee1043/.conda/envs/pmp_devel_20250722/lib/python3.10/site-packages/xcdat/regridder/regrid2.py:223: RuntimeWarning: invalid value encountered in divide
  np.divide(
2025-08-28 09:41:01,323 [ERROR]: 2831988338.py(process_references:46) >> Error for hfls TropFlux-1-0: local variable 'levels_to_plot' referenced before assignment


regrid done, ds_ac_interp[hfls].shape: (12, 72, 144)
data_var, da.shape: hfls (12, 72, 144)
Error logged for hfls TropFlux-1-0
Error from process_references for hfls TropFlux-1-0: local variable 'levels_to_plot' referenced before assignment
var: hfns
levels: [None]
=== var, ref: hfns, TropFlux-1-0
Processing data for: TropFlux-1-0
jwlee123, ref, data_path: TropFlux-1-0 /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ESSO/TropFlux-1-0/mon/hfns/gn/v20210727/hfns_mon_TropFlux-1-0_PCMDI_gn_197901-201707.nc
Processing ref dataset - varname: hfns, data: TropFlux-1-0, path: /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ESSO/TropFlux-1-0/mon/hfns/gn/v20210727/hfns_mon_TropFlux-1-0_PCMDI_gn_197901-201707.nc
ver: v20250828
ver: v20250828
get_annual_cycle, var: hfns
data_path: /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ESSO/TropFlux-1-0/mon/hfns/gn/v20210727/hfns_mon_TropFlux-1-0_PCMDI_gn_197901-201707.nc
out_path: /global/cfs/projectdirs/m4581/PMP/pmp_reference/obs4MIPs_cli

...

regrid done, ds_ac_interp[rt].shape: (12, 72, 144)
data_var, da.shape: rt (12, 72, 144)
Error logged for rt CERES-EBAF-4-2
Error from process_references for rt CERES-EBAF-4-2: local variable 'levels_to_plot' referenced before assignment
var: sfcWind
levels: [None]
=== var, ref: sfcWind, ERA-INT
Processing data for: ERA-INT
jwlee123, ref, data_path: ERA-INT /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ECMWF/ERA-INT/mon/sfcWind/gn/v20210727/sfcWind_mon_ERA-INT_PCMDI_gn_197901-201903.nc
Processing ref dataset - varname: sfcWind, data: ERA-INT, path: /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ECMWF/ERA-INT/mon/sfcWind/gn/v20210727/sfcWind_mon_ERA-INT_PCMDI_gn_197901-201903.nc
ver: v20250828
ver: v20250828
get_annual_cycle, var: sfcWind
data_path: /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ECMWF/ERA-INT/mon/sfcWind/gn/v20210727/sfcWind_mon_ERA-INT_PCMDI_gn_197901-201903.nc
out_path: /global/cfs/projectdirs/m4581/PMP/pmp_reference/obs4MIPs_clims_1980-2014/sfcWind/

2025-08-28 09:56:57,048 [ERROR]: 2831988338.py(process_references:46) >> Error for sfcWind ERA-INT: local variable 'levels_to_plot' referenced before assignment


regrid done, ds_ac_interp[sfcWind].shape: (12, 72, 144)
data_var, da.shape: sfcWind (12, 72, 144)
Error logged for sfcWind ERA-INT
Error from process_references for sfcWind ERA-INT: local variable 'levels_to_plot' referenced before assignment
=== var, ref: sfcWind, REMSS-PRW-v07r01
Processing data for: REMSS-PRW-v07r01
jwlee123, ref, data_path: REMSS-PRW-v07r01 /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/RSS/REMSS-PRW-v07r01/mon/sfcWind/gn/v20220510/sfcWind_mon_REMSS-PRW-v07r01_PCMDI_gn_198801-201812.nc
Processing ref dataset - varname: sfcWind, data: REMSS-PRW-v07r01, path: /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/RSS/REMSS-PRW-v07r01/mon/sfcWind/gn/v20220510/sfcWind_mon_REMSS-PRW-v07r01_PCMDI_gn_198801-201812.nc
ver: v20250828
ver: v20250828
get_annual_cycle, var: sfcWind
data_path: /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/RSS/REMSS-PRW-v07r01/mon/sfcWind/gn/v20220510/sfcWind_mon_REMSS-PRW-v07r01_PCMDI_gn_198801-201812.nc
out_path: /global/cfs/project

2025-08-28 09:57:34,797 [ERROR]: 2831988338.py(process_references:46) >> Error for sfcWind REMSS-PRW-v07r01: local variable 'levels_to_plot' referenced before assignment


regrid done, ds_ac_interp[sfcWind].shape: (12, 72, 144)
data_var, da.shape: sfcWind (12, 72, 144)
Error logged for sfcWind REMSS-PRW-v07r01
Error from process_references for sfcWind REMSS-PRW-v07r01: local variable 'levels_to_plot' referenced before assignment
var: ta
levels: [None]
=== var, ref: ta, ERA-40
Processing data for: ERA-40
jwlee123, ref, data_path: ERA-40 /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ECMWF/ERA-40/mon/ta/gn/v20210727/ta_mon_ERA-40_PCMDI_gn_195709-200208.nc
Processing ref dataset - varname: ta, data: ERA-40, path: /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ECMWF/ERA-40/mon/ta/gn/v20210727/ta_mon_ERA-40_PCMDI_gn_195709-200208.nc
ver: v20250828
ver: v20250828
get_annual_cycle, var: ta
data_path: /global/cfs/projectdirs/m4581/obs4MIPs/obs4MIPs_LLNL/ECMWF/ERA-40/mon/ta/gn/v20210727/ta_mon_ERA-40_PCMDI_gn_195709-200208.nc
out_path: /global/cfs/projectdirs/m4581/PMP/pmp_reference/obs4MIPs_clims_1980-2014/ta/gn
outfilename_head: ta_mon_ERA-40_PCMDI_

In [ ]:
print_dict(refs_dict)

In [ ]:
print_dict(models_dict)